In [5]:
from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/.env')

True

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index import (
    SimpleDirectoryReader,
    ServiceContext,
    get_response_synthesizer,
)
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI

In [26]:
!pip install arxiv

In [21]:
import arxiv
import datetime

def search_arxiv(keywords, max_results=10, max_age_years=5):
    """
    Search for papers on arXiv.org matching the given keywords, no older than a specified number of years.

    Args:
    keywords (str): Keywords to search for.
    max_results (int): Maximum number of results to return.
    max_age_years (int): Maximum age of papers in years.

    Returns:
    list: Result list of the resulting papers.
    """
    # Define the search query
    query = ' AND '.join(f'all:"{keyword}"' for keyword in keywords)

    # Calculate the date threshold
    date_threshold = datetime.date.today() - datetime.timedelta(days=365 * max_age_years)

    # Construct the default API client.
    client = arxiv.Client()

    # Search arXiv
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    results = client.results(search)
    print(results)

    # Fetch results and filter by age
    papers = []
    for result in results:
        if result.published.date() >= date_threshold:
            print(result)
            papers.append(result)

    return papers

# Example usage
keywords = ["phages", "machine learning"]
papers= search_arxiv(keywords, max_age_years=10)
for p in papers:
    print(p)


SSLError: HTTPSConnectionPool(host='export.arxiv.org', port=443): Max retries exceeded with url: /api/query?search_query=all%3A%22phages%22+AND+all%3A%22machine+learning%22&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1002)')))

In [33]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=24365f4bbf3605ead7845d478010b4b999a315d15a4242234bbd048d3467eb8a
  Stored in directory: /home/vscode/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [34]:
import wikipedia

title='P1 Phage'
wikipedia.search('P1 Phage')

['P1 phage',
 'Bacteriophage',
 'P1',
 'Cre-Lox recombination',
 'Nat Sternberg',
 'Cloning vector',
 'Site-specific recombination',
 'P1-derived artificial chromosome',
 'Pseudomonas virus phi6',
 'Ff phages']

In [39]:
page = wikipedia.page(title)
page.references

['http://www.expasy.org/viralzone/all_by_species/505.html',
 'https://zenodo.org/record/1235061',
 'https://ui.adsabs.harvard.edu/abs/1990PNAS...87.8070S',
 'https://api.semanticscholar.org/CorpusID:28785596',
 'https://www.irmng.org/aphia.php?p=taxdetails&id=11459714',
 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC523184',
 'https://doi.org/10.1128%2FJB.186.21.7032-7068.2004',
 'https://www.worldcat.org/issn/0021-9193',
 'https://pubmed.ncbi.nlm.nih.gov/15489417',
 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC256520',
 'https://doi.org/10.1128%2FJVI.45.3.1118-1139.1983',
 'https://www.worldcat.org/issn/0022-538X',
 'https://pubmed.ncbi.nlm.nih.gov/6834479',
 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC206102',
 'https://doi.org/10.1128%2Fjb.174.12.3936-3944.1992',
 'https://pubmed.ncbi.nlm.nih.gov/1534556',
 'https://doi.org/10.1016%2F0022-2836(92)90623-R',
 'https://www.worldcat.org/issn/0022-2836',
 'https://pubmed.ncbi.nlm.nih.gov/1324323',
 'https://doi.org/10.1016%2F0092-8674

In [41]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 3.3 MB/s eta 0:00:00a 0:00:01


In [1]:
# https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader.html

from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="/workspaces/ml-learning/src/phages/data/P1")
documents = reader.load_data()

In [9]:
# extracting metadata

# to extract reference as well? or maybe do it after the nodes and documents are created?
import mimetypes
import os
from datetime import datetime
from pathlib import Path
from typing import Any, Callable, Dict, Generator, List, Optional, Type

from pathlib import Path
def default_file_metadata_func(file_path: str) -> Dict:
    """Get some handy metadate from filesystem.

    Args:
        file_path: str: file path in str
    """
    return {
        "file_path": file_path,
        "file_name": os.path.basename(file_path),
        "file_type": mimetypes.guess_type(file_path)[0],
        "file_size": os.path.getsize(file_path),
        "creation_date": datetime.fromtimestamp(
            Path(file_path).stat().st_ctime
        ).strftime("%Y-%m-%d"),
        "last_modified_date": datetime.fromtimestamp(
            Path(file_path).stat().st_mtime
        ).strftime("%Y-%m-%d"),
        "last_accessed_date": datetime.fromtimestamp(
            Path(file_path).stat().st_atime
        ).strftime("%Y-%m-%d"),
    }

def get_meta(file_path):
    return {"foo": "bar", "file_path": file_path}


# e.g. SimpleDirectoryReader(input_dir="path/to/directory", file_metadata=get_meta)

In [10]:
# LLM
llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024)

In [11]:
import tqdm as notebook_tqdm

In [52]:
one = [d for d in documents if d.metadata['file_name'] == '0343-04.pdf']

In [54]:
len(one)

37

In [12]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: JOURNAL OF BACTERIOLOGY , Nov. 2004, p. 7032–70...
> Adding chunk: JOURNAL OF BACTERIOLOGY , Nov. 2004, p. 7032–70...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Whereas /H9261prophage leads an essentially pas...
> Adding chunk: Whereas /H9261prophage leads an essentially pas...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: out in vitro, P1-based vectors are now in commo...
> Adding chunk: out in vitro, P1-based vectors are now in commo...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: MATERIALS AND METHODS
Sources of DNA and sequen...
> Adding chunk: MATERIALS AND METHODS
Sources of DNA and sequen...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: The assignment of previously identi ﬁed genetic...
> Adding chunk: The assignment of previously identi ﬁed genetic...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: numbers AF234172 (phage P1 mod749 ::IS5c1-100) ...
> Adding chunk: n

> Adding chunk: RecBCD in P1 cyclization could explain the 10-f...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: The two octamers that are the most frequent in ...
> Adding chunk: The two octamers that are the most frequent in ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: It, iteron.
bThe ﬁrst number is the coordinate ...
> Adding chunk: It, iteron.
bThe ﬁrst number is the coordinate ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: as the most infrequent tetramer in E. coli (35)...
> Adding chunk: as the most infrequent tetramer in E. coli (35)...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: hot 85.3385.6 ppfA 62.3462.7
12 SeeR? hrdC 64.1...
> Adding chunk: hot 85.3385.6 ppfA 62.3462.7
12 SeeR? hrdC 64.1...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: pacA 89.3390.5 trnT 68.2368.3
c6 See lxc pacB 9...
> Adding chunk: pacA 89.3390.5 trnT 68.2368.3
c6 See lxc pacB 9...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: T

In [16]:
# https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/root.html           
query_engine = index.as_query_engine(streaming=True)
query_str = "What is the morphology of the P1 Phage?"
streaming_response = query_engine.query("What is the morphology of the P1 Phage?")
streaming_response.print_response_stream()

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f91300>, 'json_data': {'input': ['What is the morphology of the P1 Phage?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f91300>, 'json_data': {'input': ['What is the morphology of the P1 Phage?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
connect_tcp.started host='api.openai.com' port=443 local_address=None t

In [14]:
response = streaming_response.get_response()

In [15]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='5bb7a454-99ef-4460-8596-902316659ca4', embedding=None, metadata={'page_label': '22', 'file_name': 'jvirol00150-0228.pdf', 'file_path': '/workspaces/ml-learning/src/phages/data/P1/jvirol00150-0228.pdf', 'file_type': 'application/pdf', 'file_size': 6891222, 'creation_date': '2024-01-05', 'last_modified_date': '2024-01-05', 'last_accessed_date': '2024-01-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b652dc39-a92b-4c58-847a-dd5b4c870838', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '22', 'file_name': 'jvirol00150-0228.pdf', 'file_path': '/workspaces/ml-learning/src/phages/data/P1/jvirol00150-0228.pdf', 'file_type': 'application/pdf', 'file_size': 68912

In [18]:
from llama_index.query_engine import CitationQueryEngine
citation_query_engine = CitationQueryEngine.from_args(index)
response = citation_query_engine.query(query_str)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f92160>, 'json_data': {'input': ['What is the morphology of the P1 Phage?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f92160>, 'json_data': {'input': ['What is the morphology of the P1 Phage?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f92160>, 'json_data': {'input': ['What is the morphology of the P1 Phage?'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encodin

In [20]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))

<b>The morphology of the P1 phage is characterized by an icosahedral head attached to a tail that has six kinked tail fibers [4].</b>

In [25]:
print(response.response)
print(response.source_nodes)
print(response.get_formatted_sources(length=500))

The morphology of the P1 phage is characterized by an icosahedral head attached to a tail that has six kinked tail fibers [4].
[NodeWithScore(node=TextNode(id_='5bb7a454-99ef-4460-8596-902316659ca4', embedding=None, metadata={'page_label': '22', 'file_name': 'jvirol00150-0228.pdf', 'file_path': '/workspaces/ml-learning/src/phages/data/P1/jvirol00150-0228.pdf', 'file_type': 'application/pdf', 'file_size': 6891222, 'creation_date': '2024-01-05', 'last_modified_date': '2024-01-05', 'last_accessed_date': '2024-01-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b652dc39-a92b-4c58-847a-dd5b4c870838', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '22', 'file_name': 'jvirol00150-0228.pdf', 'fi

AttributeError: 'Response' object has no attribute 'response_txt'

In [34]:
response = citation_query_engine.query("You are an expert writing a scientific review paper. Please provide a summary section written in scientific language for specialist that summarize the morphology of the P1 Phage based on the information in the context. Please reference each statement with a citation. Then display a list of citations.")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f7fd80>, 'json_data': {'input': ['You are an expert writing a scientific review paper. Please provide a summary section written in scientific language for specialist that summarize the morphology of the P1 Phage based on the information in the context. Please reference each statement with a citation. Then display a list of citations.'], 'model': <OpenAIEmbeddingModeModel.TEXT_EMBED_ADA_002: 'text-embedding-ada-002'>, 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7fde74f7fd80>, 'json_data': {'input': ['You are an expert writing a scientific review paper. Please provide a summary section written in scientific language for specialist that summarize the morphology of the P1 Phage based on the information 

In [35]:
display(Markdown(f"<b>{response}</b>"))

<b>The morphology of the P1 phage has been extensively studied [2][47][48][49][50]. It has been observed that the P1 phage exhibits morphological variants [47]. The assembly of the P1 phage tail fibers has been investigated, and tail fiber precursors have been isolated and characterized [52][53]. The genetic control of complex bacteriophage assembly has also been explored [53]. The genome of the P1 phage has been analyzed, providing insights into its structure and organization [54]. Additionally, specific antibody association has been used to determine the positions of gene products in the P1 phage [55]. These studies have contributed to our understanding of the morphology and genetic characteristics of the P1 phage.

List of Citations:
1. Source 2: Sternberg, N., Hamilton, D., Austin, S., Yarmolinsky, M., & Hoess, R. (1981). Site-specific recombination and its role in the lifecycle of bacteriophage P1. Cold Spring Harbor Symp Quant Biol, 45, 297-309.
2. Source 2: Toussaint, A., Lefebre, N., Scott, J. R., Cowan, J. A., Bruijn, F. D., & Bukhari, A. I. (1978). Relationships between temperate phages Mu and P1. Virology, 89, 146-161.
3. Source 2: vandePutte, P., Cramer, S., & Giphart-Gassler, M. (1980). Invertible DNA determines host specificity of bacteriophage Mu. Nature (London), 286, 218-222.
4. Source 2: Walker, D. H., Jr., & Anderson, T. F. (1970). Morphological variants of coliphage P1. J. Virol., 5, 765-782.
5. Source 2: Walker, D. H., Jr., & Walker, J. T. (1975). Genetic studies of coliphage P1. I. Mapping by use of prophage deletions. J. Virol., 16, 525-534.
6. Source 2: Walker, D. H., Jr., & Walker, J. T. (1976). Genetic studies of coliphage P1. III. Extended genetic map. J. Virol., 20, 177-187.
7. Source 2: Walker, J. T., & Walker, D. H., Jr. (1980). Mutations in coliphage P1 affecting host cell lysis. J. Virol., 35, 519-530.
8. Source 2: Walker, J. T., & Walker, D. H., Jr. (1981). Structural proteins of coliphage P1. In M. S. DuBow (Ed.), Progress in clinical biological research, vol. 64 (pp. 69-77). Alan R. Liss, Inc., New York.
9. Source 3: Wood, W. B., & King, J. (1979). Genetic control of complex bacteriophage assembly. Compr. Virol., 13, 581-633.
10. Source 3: Wood, W. B., & Revel, H. R. (1976). The genome of bacteriophage T4. Bacteriol. Rev., 40, 847-868.
11. Source 3: Yanagida, M., & Ahmad-Zadeh, C. (1970). Determination of gene product positions in bacteriophage T4 by specific antibody association. J. Mol. Biol., 51, 411-421.
12. Source 3: Yanagida, M., BoydelaTour, E., Alff-Steinberger, C., & Kellenberger, E. (1970). Studies on the morphopoiesis of the head of bacteriophage T-even. VIII. Multilayered polyheads. J. Mol. Biol., 50, 35-58.
13. Source 3: Yarmolinsky, M. (1982). Bacteriophage P1. In S. J. O'Brien (Ed.), Genetic maps, vol. 2 (pp. 34-43). National Cancer Institute, Bethesda, Md.</b>

In [44]:
# # default mode of building the index
# response_synthesizer = get_response_synthesizer(
#     response_mode="tree_summarize", use_async=True
# )
# doc_summary_index = DocumentSummaryIndex.from_documents(
#     documents,
#     service_context=service_context,
#     response_synthesizer=response_synthesizer,
#     show_progress=True,
# )

/workspaces/ml-learning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarizing documents:   0%|          | 0/229 [00:00<?, ?it/s]

current doc id: f5f7f7fa-761a-4751-a481-63179be9aaef


Summarizing documents:   0%|          | 1/229 [00:15<57:17, 15.08s/it]

current doc id: dc37410d-bede-4727-8cc6-36942300a33c


Summarizing documents:   1%|          | 2/229 [00:25<47:45, 12.63s/it]

...

current doc id: 708590ea-daef-4561-8a67-3e57d3f5f0ea


Summarizing documents:  16%|█▌        | 36/229 [08:45<1:37:22, 30.27s/it]

current doc id: 7936a034-66de-4f84-8180-0bb8af233c68


Summarizing documents:  16%|█▌        | 36/229 [09:06<48:50, 15.18s/it]  


KeyboardInterrupt: 